In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, fig_config, set_axes_equal,
                              save_fig)
from helpers import (clean_df, export_pcd, export_fields, poynting_vector,
                     export_rect_idx, get_imcolors)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# CAD model

In [ ]:
# input data

polarization = 'te'
f = 26

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()

In [ ]:
# generate a point cloud

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
pcd.paint_uniform_color([0.5, 0.5, 0.5])
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
pcd.estimate_normals()
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

In [ ]:
view_config = {
    'zoom': 0.6,
    'front': [0.99757547016811021, -0.0053197730130150481, 0.069389345968699973],
    'lookat': [73.970504527733638, 21.162127098201477, -9.8491774343806799],
    'up': [0.0062516025708518452, 0.99989308520692732, -0.01321876017899459],
}
color = get_imcolors([pcd, rec_mesh], view_config)
set_colorblind()
fig_config()
fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(color, origin='upper')

# fname = os.path.join('figures', 'ear_model')
# save_fig(fig, fname=fname, formats=['png'])